# Capítulo 04: Metadados e Snapshots

Este notebook explora metadados e histórico de snapshots em tabelas Iceberg.

**Objetivo:** Entender e utilizar metadados para análises e auditoria.

**Conteúdo:**
- Exploração de metadados de tabelas
- Histórico de snapshots
- Estatísticas de arquivos
- Visualização de evolução de dados

## 6. Resumo

Neste capítulo você aprendeu:
- ✅ Como explorar metadados de tabelas Iceberg
- ✅ Visualizar histórico de snapshots
- ✅ Calcular estatísticas de arquivos
- ✅ Criar ferramentas de análise reutilizáveis

**Próximo:** Capítulo 05 - Time Travel e Versionamento

In [ ]:
class IcebergMetadataExplorer:
    """Explora metadados de tabelas Iceberg"""
    
    def __init__(self, table_path):
        self.table_path = table_path
        self.con = duckdb.connect()
        safe_install_ext(self.con, "iceberg")
    
    def get_file_stats(self):
        """Estatísticas de arquivos"""
        try:
            return self.con.execute(f"""
                SELECT
                    count(*) as file_count,
                    sum(record_count) as total_records,
                    sum(file_size_in_bytes) / 1024 / 1024 as total_mb
                FROM iceberg_metadata('{self.table_path}')
                WHERE status = 'EXISTING'
            """).fetchone()
        except Exception as e:
            print(f"❌ Erro: {e}")
            return None
    
    def get_snapshot_history(self):
        """Histórico de snapshots"""
        try:
            return self.con.execute(f"""
                SELECT *
                FROM iceberg_snapshots('{self.table_path}')
                ORDER BY sequence_number DESC
            """).df()
        except Exception as e:
            print(f"❌ Erro: {e}")
            return None
    
    def get_latest_snapshot(self):
        """Último snapshot"""
        try:
            return self.con.execute(f"""
                SELECT snapshot_id, timestamp_ms
                FROM iceberg_snapshots('{self.table_path}')
                ORDER BY sequence_number DESC
                LIMIT 1
            """).fetchone()
        except Exception as e:
            print(f"❌ Erro: {e}")
            return None

# Exemplo de uso
# explorer = IcebergMetadataExplorer('s3://bucket/sales')
# stats = explorer.get_file_stats()
print("✅ Classe IcebergMetadataExplorer criada!")

## 5. Classe Explorer de Metadados

In [ ]:
# Estatísticas completas
query_stats = """
SELECT
    count(*) as total_files,
    sum(record_count) as total_records,
    sum(file_size_in_bytes) / 1024 / 1024 / 1024 as total_size_gb,
    avg(file_size_in_bytes) / 1024 / 1024 as avg_file_size_mb,
    min(record_count) as min_records,
    max(record_count) as max_records
FROM iceberg_metadata('s3://bucket/sales')
WHERE status = 'EXISTING'
"""

print("Query de estatísticas:")
print(query_stats)

## 4. Estatísticas da Tabela

In [ ]:
# Ver histórico de snapshots
query_snapshots = """
SELECT
    snapshot_id,
    to_timestamp(timestamp_ms / 1000) as snapshot_time,
    sequence_number
FROM iceberg_snapshots('s3://bucket/sales')
ORDER BY sequence_number DESC
"""

print("Query para listar snapshots:")
print(query_snapshots)
print("\nℹ️  Esta query mostra todas as versões da tabela")

## 3. Histórico de Snapshots

Cada modificação cria um novo snapshot.

In [ ]:
# Explorar metadados (exemplo demonstrativo)
query_metadados = """
SELECT
    file_path,
    record_count,
    file_size_in_bytes / 1024 / 1024 as size_mb,
    status,
    content
FROM iceberg_metadata('s3://bucket/sales')
"""

print("Query para explorar metadados:")
print(query_metadados)
print("\nℹ️  Use esta query ajustando o caminho da sua tabela")

## 2. Explorar Metadados da Tabela

Os metadados revelam informações sobre arquivos, tamanhos e estatísticas.

In [ ]:
# Importar bibliotecas
import duckdb
import pandas as pd
from datetime import datetime
import importlib.util


def safe_install_ext(con, ext_name):
    try:
        con.execute(f"INSTALL {ext_name}")
        con.execute(f"LOAD {ext_name}")
        print(f"✅ {ext_name} carregado")
        return True
    except Exception as e:
        print(f"⚠️  Erro: {e}")
        return False


# Setup
con = duckdb.connect()
safe_install_ext(con, "iceberg")
print("✅ Ambiente configurado!")

In [ ]:
# Instalar dependências
import sys
!{sys.executable} -m pip install -q duckdb pyarrow pandas matplotlib

## 1. Setup Inicial